In [5]:
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import\
    DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import P
from gremlin_python.process.graph_traversal import __
import json
import uuid
from gremlin_python.process.traversal import Order
import nest_asyncio
nest_asyncio.apply()
from datetime import datetime
from gremlin_python.driver.client import Client
import logging
import time
from typing import List
from gremlin_python.process.traversal import Column

In [6]:
server = '135.181.30.253'
port = 8182
endpoint = 'ws://' + server + ':' + str(port) + '/gremlin'
transport_args = {'max_content_length': 200000}

connection = DriverRemoteConnection(endpoint, 'g', **transport_args)
# Connection 1 to read data
g = traversal().withRemote(connection)
# Connection 2 to write data
client = Client(endpoint, 'g')


In [7]:
# Print 10 messages from server
g.V().limit(10).valueMap().toList()

[{'id': [959233366],
  'created_at': ['07/21/2021, 10:27:17'],
  'text': ['@BDliveSA I stand with honesty and I always will promote honesty. @Ethans_Rolands You are indeed a honest person. H… https://t.co/4M6vRcsQRC'],
  'source': ['<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>'],
  'polarity': ['0.6'],
  'subjectivity': ['0.9'],
  'assessments': ['[[[honest], 0.6, 0.9, null]]']},
 {'id': [1505115756],
  'created_at': ['07/21/2021, 10:28:12'],
  'text': ['RT @Allowancefi: 🚀Huge Airdrop to promote Allowance🎉\n\n💰10,000,000 Allowance for 50 lucky participants. \n💰10,000 Allowance for first 10,000…'],
  'source': ['<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>'],
  'polarity': ['0.29166666666666663'],
  'subjectivity': ['0.5833333333333334'],
  'assessments': ['[[[lucky], 0.3333333333333333, 0.8333333333333334, null], [[first], 0.25, 0.3333333333333333, null]]']},
 {'id': [369509540],
  'created_at': ['07/21/202

In [9]:
# drop all messages from the server
# g.V().drop().iterate()
# g.V().count().next()

20000

In [ ]:
# Command to check information from the server
# docker-compose -f  docker-compose-cql-es.yml logs -f

# Parse data to server


In [10]:
def create_inject_query(message, node):
    return g.inject([message])\
        .unfold().as_('entity')\
        .addV(node).as_('v')\
        .sideEffect(__.select('entity')
                    .unfold().as_('kv').select('v')
                    .property(__.select('kv')
                              .by(Column.keys),
                              __.select('kv')
                              .by(Column.values))
                    )\
        .id()\
        .toList()

In [11]:
## create message 
message = {
    'text': 'hello, this is test message',
    'polarity': '1.0'
}

In [13]:
## send to server message
create_inject_query(message=message, node='tweet')

[7987440]

In [15]:
# check message on the server
g.V('7987440').valueMap().toList()

[{'text': ['hello, this is test message'], 'polarity': [1.0]}]

# To send big data messages to server. Aproach 2

In [21]:

data = '[["crypto": "BTC"], ["crypto": "ETH", "coef": 2]]'
node = 'market'
req = f'g.inject({data}).unfold().as("m").addV("{node}").as("v").select("m").unfold().as("kv").select("v").property(select("kv").by(keys), select("kv").by(values)).id().toList()' 
client.submit(req).all().result()

[16425160, 8495312, 8495312]

In [37]:
g.V(8495312).label().toList()

['market']

# Generate data


In [8]:
import numpy as np
crypto_tickers = ['BTC', 'ETH', 'LIT', 'SFP', 'TWT', 'DOT']
trading_signals = ['buy', 'sel', 'hold']
social_activites = ['high', 'slow', 'normal']
coefs = np.random.random_integers(low=1, high=100, size=20)
messages = list()
for i in range(10):
    ticker = np.random.choice(crypto_tickers)
    signal = np.random.choice(trading_signals)
    activities = np.random.choice(social_activites)
    coef = np.random.choice(coefs)
    # Test for generate data
    # combine_data = dict({
    #     'ticker': ticker,
    #     'signal': signal,
    #     'activities': activities,
    #     'coef': coef
    # })
    # messages.append(combine_data)

    # Send data to server
    data = f'[["ticker": "{ticker}", "signal": "{signal}", "activities": "{activities}", "coef": "{coef}"]]'
    node = 'cryptos'
    req = f'g.inject({data}).unfold().as("m").addV("{node}").as("v").select("m").unfold().as("kv").select("v").property(select("kv").by(keys), select("kv").by(values)).id().toList()'
    client.submit(req).all().result()


<ipython-input-8-ab0a4563e353>:5: DeprecationWarning: This function is deprecated. Please call randint(1, 100 + 1) instead
  coefs = np.random.random_integers(low=1, high=100, size=20)


In [9]:
g.V().hasLabel(node).count().toList()

[10]

In [10]:
# Architecture to parse data
# Parse data
# 1 Twiiter
# 2. Telegram 
# 3. News
# 4. Binance api (transaction, volume, count ticks)
# datetime.now()